**Objetivo: K-Means y Jerarquico "Complete"**

Se reclasificará la información en *clusters* usando los métodos Jerarquico *Complete* y *Single* de *Machine Learning *(ML), no supervisado, y se compraran las diferencias entre los resultados.

La [base de datos](https://archive.ics.uci.edu/ml/datasets/Heart+Disease) tiene información sobre pacientes de la ciudad de Cleveland, con  variables usadas en varios estudios para determinar si un paciente tiene una enfermedad del corazón y finalmente el diagnostico de enfermedad cardiaca.


In [ ]:
#Importación de las librerías de interés
import numpy as np                # linear algebra
import pandas as pd               # data frames
import seaborn as sns             # visualizations
import matplotlib.pyplot as plt   # visualizations
import scipy.stats                # statistics

from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.cluster.hierarchy import complete, fcluster #Cluster Jerarquico Completo

from sklearn import preprocessing
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

import os
print(os.listdir("../input"))

In [ ]:
#Se renombra la base de datos a trabajar y se realiza la exploración inicial
df = pd.read_csv("../input/heart.csv")

# Print the head of df
print(df.head())
# Print the info of df
print(df.info())
# Print the shape of df
print(df.shape)

**Resultado primer acercamiento a la base:** Información para 303 pacientes con 14 variables numericas, sin información faltante. Las variables relacionadas son: 

**age**: edad en años
**sex:** 0 para hombre y 1 para mujer
**cp:** tipo de dolor en el pecho (0, 1, 2, 3 y 4:  typical angina, atypical angina, non-anginal pain y asymptomatic. No dice, que número representa que tipo de dolor, pero se asumen en el orden en el que estan)
**trestbps**: Presión arterial en reposo (en mm Hg al ingreso en el hospital)
**chol**: Colesterol sérico en mg / dl
**fbs:** 1: Azúcar en sangre ayunas >120 mg/dl y 0: Azúcar en sangre ayunas <120 mg/dl
**restecg**:Resultados electrocardiográficos en reposo. ( 0: normal,  1: anomalía de la onda ST-T y 2: muestra la hipertrofia ventricular izquierda probable o definitiva según los criterios de Estes)
**thalach**: ritmo cardíaco máximo alcanzado
**exang**: angina inducida por el ejercicio (1 = sí; 0 = no)
**oldpeak**:Depresión ST inducida por el ejercicio en relación con el descanso.
**slope**:La pendiente del segmento pico del ejercicio ST. (0: pendiente ascendente, 1: plano y 2: bajada)
**ca**: número de vasos principales (0-3) coloreados por fluoroscopia 
**thal:** 3 = normal; 6 = defecto fijo; 7 = defecto reversible (no se corresponde con los códigos de la base.
**target:** Diagnóstico de enfermedad cardíaca (estado de enfermedad angiográfica; 0: <50% de diámetro de estrechamiento y 1: > 50% de diámetro de estrechamiento.)

***Nota:*** La descripción de algunas variables no se corresponde con sus valores en la base.


**Analisis Exploratorio de Datos Básico**

In [ ]:
#Médidas básicas para todas las variables
df.describe()

In [ ]:
#¿La base está balaceada?
df.isnull().sum()

In [ ]:
#histograma para variables continuas
f, axes = plt.subplots(2,2, figsize=(20, 12))
f.suptitle("Distribución variables continuas", fontsize=20)
sns.distplot( df["age"], ax=axes[0,0])
sns.distplot( df["chol"], ax=axes[0,1])
sns.distplot( df["thalach"], ax=axes[1,0])
sns.distplot( df["oldpeak"], ax=axes[1,1])

Las variables Edad y colesterol parecen tener una distribución más o menos normal,thalach (ritmo cardíaco máximo alcanzado) parece tener un ligero sesgo a la derecha y claramente oldpeak (Depresión ST inducida por el ejercicio en relación con el descanso) se consentra alrededor de cero, indicando que la poca tolerancia al esfuerzo sobresale en los pacientes. Se resalta que la media de la edad se ubica alrededor de los 55 años.

In [ ]:
#Histogramas para las variables categoricas 
fig, axes = plt.subplots(nrows=2, ncols=4, figsize=(18, 10))
fig.suptitle("Distribución de frecuencias para las variables categoricas", fontsize=20)

x = ['Mujer','Hombre'];y = df.sex.value_counts(sort = False).values
axes[0][0].bar(x,y);axes[0][0].set_title('Sexo')

x = ['Typical','Atypical','Non-Anginal','Aysyptomatic'];y = df.cp.value_counts(sort = False).values
axes[0][1].bar(x,y);axes[0][1].set_title('Dolor en el Pecho-cp')

x = ['<120 mg/dl','>120 mg/dl'];y = df.fbs.value_counts(sort = False).values
axes[0][2].bar(x,y);axes[0][2].set_title('Azúcar sangre ayunas-fbs')

x = ['Regular','Abnormality','Severe'];y = df.restecg.value_counts(sort = False).values
axes[0][3].bar(x,y);axes[0][3].set_title('Electrocardiograma-restecg')

x = ['No','Si'];y = df.exang.value_counts(sort = False).values
axes[1][0].bar(x,y);axes[1][0].set_title('Angina por ejercicio-exang')

x = ['Downward','Flat','Upward'];y = df.slope.value_counts(sort = False).values
axes[1][1].bar(x,y);axes[1][1].set_title('ST excercise peak-slope')

x = ['None','Normal','Fixed Defect','Reversable Defect'];y = df.thal.value_counts(sort = False).values
axes[1][2].bar(x,y);axes[1][2].set_title('Thalium Stress Test - Thal')

x = ['No','Si'];y = df.target.value_counts(sort = False).values
axes[1][3].bar(x,y);axes[1][3].set_title('Enfermedad Coronaria - Target')

plt.show()

**Los histogramas nos dan mucha información de las variables categoricas:**

La incidencia de la enfermedad coronaria (Target) afecta a casi la mitad de la población (46%) y la mayoría de los pacientes son mujeres. Asimismo, el dolor de pecho (cp) más frecuente en la "tipica angina", seguido por "no dolor anginal". Por su parte las anginas no son producto del ejercicio. 

En cuanto a los recursos diagnosticos, la mayoría de los pacientes presentan defectos fijos o reversibles (thalium stress-thal);  picos de esfuerzo planos o al alza (ST excercise peak-slope) y electrocardigramas no severos. Curiosamente no tienen azúcar alta, parece que esta variable tiene poca incidencia en los pacientes cardiacos.

No obstante, los histogramas no nos dicen cómo se relacionan estas variables con tener o no una enfermedad cardiaca. 

In [ ]:
#los nombres no dicen mucho, entonces se renombraran
df=df.rename(columns={'age':'Edad','sex':'Sexo','cp':'Dolor_pecho','trestbps':'Presión','chol':'Colesterol','fbs':'Azúcar >120','restecg':'Electro','thalach':'Ritmo_cardiaco','exang':'Angina_ejercicio','oldpeak':'Depresión_ST ','slope':'Pico_ejercicio','ca':'Vasos_Ppales','thal':'Defecto','target':'Diagnostico'})
df.columns

In [ ]:
# Compute the correlation matrix
corr=df.corr()

# Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=np.bool);mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, center=0,square=True, linewidths=.5, cbar_kws={"shrink": .5})

Al parecer las variables más relacionadas con tener una enfermedad cardiaca son el tipo de dolor en el pecho (cp), el ritmo cardiaco (thalach) y la relación de pendiente entre el pico del ejercicio y el descanso (slope). Gráfiquemos la relación entre variables.

In [ ]:
#Grafico de violin
f, axes = plt.subplots(1,2, figsize=(20, 12))
f.suptitle("Ritmo cardiaco Vs. Prueba esfuerzo y tipo de dolor de pecho", fontsize=20)

sns.catplot(x="Pico_ejercicio", y="Ritmo_cardiaco", hue="Diagnostico",kind="violin", split=True, data=df,ax=axes[0] )
sns.catplot(x="Dolor_pecho", y="Ritmo_cardiaco", hue="Diagnostico",kind="violin", split=True, data=df, ax=axes[1] ) 

Definitivamente un mayor ritmo cardiaco parace tener una mayor incidencia en una enfermedad coronaria, sobre todo si existe algún dolor en el pecho. Esto lo vemos mejor en el siguiente gráfico:

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(18, 10))
fig.suptitle("Ritmo cardiaco Vs. Prueba esfuerzo y tipo de dolor de pecho", fontsize=20)

sns.catplot(x="Dolor_pecho", y="Ritmo_cardiaco", hue="Diagnostico",kind="point", ax=axes[0],data=df )
sns.catplot(x="Pico_ejercicio", y="Ritmo_cardiaco", hue="Diagnostico",kind="point", data=df,ax=axes[1])

**Transformación de los datos**

Para poder hacer clusters es importante estandarizar primero los datos y quitar el efecto magnitud de ellos. 

In [ ]:
#Codificando las variables categoricas
df = pd.read_csv("../input/heart.csv")
#creamos variables categoricas separadas y quitamos una para no generar problemas de multicolinealidad
df_dum=pd.get_dummies(df,columns=["sex","cp","fbs","restecg","exang","slope","thal","target"],drop_first=True)
df_dum.describe()

In [ ]:
heart_scale = df_dum
scaler = preprocessing.StandardScaler()
columns =df_dum.columns
heart_scale[columns] = scaler.fit_transform(heart_scale[columns])
heart_scale.head()

**Hierarchical clustering / Complete**

Ahora que tenemos la base normalizada aplicaremos el algoritmo de clusterin herarquico completo para las variables que más se relacionan con la posibilidad de tener un infarto.

In [ ]:
#Primero generamos un dendograma y generamos la matriz de distancias "Z" que se requiere para 
#el cluster jerarquico completo

Z = linkage(heart_scale.loc[:,["thalach","cp_1","cp_2","cp_3","target_1","slope_1","slope_2"]], 'complete', metric='euclidean')
fig = plt.figure(figsize=(25, 10))
dn = dendrogram(Z)
plt.show()


In [ ]:
#Pasamos a generar los cluster, para lo cual definimos K=5 y K=3
#conforme a la información del dendograma

h_cluster=df.copy()
h_cluster['5_clust']=fcluster(Z,t=5, criterion='distance')
h_cluster['3_clust']=fcluster(Z,t=3, criterion='distance')
h_cluster.head()

In [ ]:
# Compute the correlation matrix
corr_cluster=h_cluster.corr()

# Generate a mask for the upper triangle
mask = np.zeros_like(corr_cluster, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr_cluster, mask=mask, cmap=cmap, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})



Observamos que las variables "5_clust" y "3_clust" tienen una alta correlación con el diagnostico, pero al parecer 5 clusteres tienes mayor relación.

In [ ]:
#los nombres no dicen mucho, entonces se renombraran
h_cluster=h_cluster.rename(columns={'age':'Edad','sex':'Sexo','cp':'Dolor_pecho','trestbps':'Presión','chol':'Colesterol','fbs':'Azúcar >120','restecg':'Electro','thalach':'Ritmo_cardiaco','exang':'Angina_ejercicio','oldpeak':'Depresión_ST ','slope':'Pico_ejercicio','ca':'Vasos_Ppales','thal':'Defecto','target':'Diagnostico'})
h_cluster.columns

fig, axes = plt.subplots(1,2, figsize=(18, 10))
fig.suptitle("grupos de pancientes", fontsize=20)

sns.catplot(x="Dolor_pecho", y="Ritmo_cardiaco", hue="Diagnostico",col="5_clust", aspect=.6,
            kind="swarm", data=h_cluster);

sns.catplot(x="Pico_ejercicio", y="Ritmo_cardiaco", hue="Diagnostico",col="5_clust", aspect=.6,
            kind="swarm", data=h_cluster);


**Los Clusters**

Cluster 1 = Corazones felices

Cluster 2= No te esfuerces Corazón!

Cluster 3= Corazones en riesgo controlable

cluster 4= Riesgo eventual, pero si le duele vaya a urgencias

Cluter 5 = Cuidado

**Hierarchical clustering / Single**


In [ ]:
#Primero generamos un dendograma y generamos la matriz de distancias "Z" que se requiere para 
#el cluster jerarquico sencillo

Z = linkage(heart_scale.loc[:,["thalach","cp_1","cp_2","cp_3","target_1","slope_1","slope_2"]],
            'single', metric='euclidean')
fig = plt.figure(figsize=(25, 10))
dn = dendrogram(Z)
plt.show()

Nos arrojó solo 3 clusters

In [ ]:
#Pasamos a generar los cluster, para lo cual definimos K=3
#conforme a la información del dendograma

h_single=df.copy()
h_single['3_clust']=fcluster(Z,t=5, criterion='distance')

h_single.head()

In [ ]:
# Compute the correlation matrix
corr_single=h_single.corr()

# Generate a mask for the upper triangle
mask = np.zeros_like(corr_single, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr_single, mask=mask, cmap=cmap, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

A diferencia del complete, el método single no arroja una clasificación con correlación significativa. Nos quedamos con "complete" para este caso.

**Clusters Complete**

Cluster 1 = Corazones felices

Cluster 2= No te esfuerces Corazón!

Cluster 3= Corazones en riesgo controlable

cluster 4= Riesgo eventual, pero si le duele vaya a urgencias

Cluter 5 = Cuidado